In [283]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
from glob import glob
import os
import time

In [284]:
image_dir = 'E:\\File Ngawur\\URGENNN\\Final Project CV\\Dataset\\Data'
label_dir = 'E:\\File Ngawur\\URGENNN\\Final Project CV\\Dataset\\GT'

path_save = 'E:\\File Ngawur\\URGENNN\\Final Project CV\\Dataset\\HASIL\\'

filelist = glob(os.path.join(image_dir, '*.tif'))
filelist_label = glob(os.path.join(label_dir, '*.jpg'))

In [285]:
image_list = []
label_list = []
list_nama = []

for file in filelist:
    image_list.append(cv.imread(file,1)[10:-10, 10:-10])
    list_nama.append(file[53:-4])
for file in filelist_label:
    img_gray = cv.imread(file,0)[10:-10, 10:-10]
    ret, img = cv.threshold(img_gray,150,255,cv.THRESH_BINARY)
    img = img//255
    label_list.append(img)

In [286]:
def create_mask(r):
    kernel = cv.getStructuringElement(cv.MORPH_CROSS, (3,3))
    ret, thres = cv.threshold(r,35,255,cv.THRESH_BINARY)
    thres = cv.morphologyEx(thres, cv.MORPH_OPEN, kernel)
    thres = cv.morphologyEx(thres, cv.MORPH_CLOSE, kernel)
    thres = cv.erode(thres, kernel, iterations = 1)
    return thres

In [287]:
kernel = cv.getStructuringElement(cv.MORPH_CROSS,(9,9))
clahe = cv.createCLAHE(clipLimit=3, tileGridSize=(8,8))
hasil = []
i = 0
for image in image_list :
    b, g, r = cv.split(image)
    g = clahe.apply(g)
    blur = cv.blur(g,(60,60))
    diff = cv.subtract(blur,g)
    ret, otsu = cv.threshold(diff,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
    mask = create_mask(r)
    masked_image = cv.bitwise_and(otsu,otsu,mask=mask)
    final_image = cv.morphologyEx(masked_image, cv.MORPH_OPEN, kernel)
    cv.imwrite(path_save + filelist[i][41:-4]+".jpg", final_image)
    hasil.append(final_image)
    i = i + 1

In [288]:
for i, r in enumerate(hasil) :
    hasil[i] = r//255

has = np.array(hasil)
has = has.reshape(-1, has.shape[1]*has.shape[2])
label = np.array(label_list)
label = label.reshape(-1, label.shape[1]*label.shape[2])

In [289]:
from sklearn.metrics import accuracy_score,roc_auc_score,precision_score,recall_score,f1_score,log_loss
from sklearn.metrics import confusion_matrix, classification_report

In [290]:
from prettytable import PrettyTable
tabel = PrettyTable(["Gambar","Accuracy","Sensitivity","Specificity", "Ppv", "Npv"])
i = 0
from sklearn.metrics import confusion_matrix
acc = 0
sen = 0
spe = 0
p = 0
n = 0

acc_l = []
sen_l = []
spe_l = []
p_l = []
n_l = []


for h, l in zip(has,label):
    
    tn, fp, fn, tp = confusion_matrix(l, h).ravel()
    
    accuracy = round((tp+tn)/(tp+tn+fp+fn)*100,4)
    sensitivity = round(tp/(tp+fn)*100,4)
    specificity = round(tn/(tn+fp)*100,4)
    ppv = round(tp/(tp+fp)*100,4)
    npv = round(tn/(tn+fn)*100,4)
    
    acc += accuracy
    sen += sensitivity
    spe += specificity
    p += ppv
    n += npv
    
    acc_l.append(accuracy)
    sen_l.append(sensitivity)
    spe_l.append(specificity)
    p_l.append(ppv)
    n_l.append(npv)
    
    param = [list_nama[i],accuracy,sensitivity,specificity,ppv,npv]
    tabel.add_row(param)
    i = i + 1
    print("Done ", *param)
tabel.add_row(["Average", round(acc/i,4), round(sen/i,4), round(spe/i,4), round(p/i,4), round(n/i,4)])
acc_l = np.array(acc_l)
sen_l = np.array(sen_l)
spe_l = np.array(spe_l)
p_l = np.array(p_l)
n_l = np.array(n_l)
tabel.add_row(["STD", round(np.std(acc_l),4), round(np.std(sen_l),4), round(np.std(spe_l),4), round(np.std(p_l),4), round(np.std(n_l),4)])

Done  image10prime 98.0714 83.0358 99.258 89.828 98.6692
Done  image11prime 97.3369 78.4974 98.4606 75.2557 98.7142
Done  image13prime 97.8856 81.3118 99.2783 90.4473 98.4428
Done  image14prime 98.0797 81.9453 99.594 94.986 98.327
Done  image15prime 98.3004 80.9634 99.4426 90.5375 98.7546
Done  image16prime 97.9263 75.8992 99.6369 94.198 98.1561
Done  image17prime 97.549 82.0033 99.0041 88.5161 98.3269
Done  image18prime 97.8589 82.8765 99.406 93.5095 98.2523
Done  image19prime 98.0515 78.1241 99.6061 93.9291 98.3155
Done  image1prime 98.3561 84.6174 99.3908 91.2744 98.8478
Done  image24prime 97.5521 77.0266 99.609 95.1784 97.7409
Done  image27prime 98.1271 84.0194 99.281 90.5283 98.7005
Done  image2prime 98.1483 82.3692 99.2523 88.5151 98.7725
Done  image3prime 98.6364 82.569 99.4567 88.5834 99.1131
Done  image4prime 97.8474 79.9937 99.1874 88.0787 98.5087
Done  image5prime 98.253 83.5113 99.2398 88.0286 98.9001
Done  image6prime 97.6133 81.3531 99.1982 90.8171 98.2008
Done  image7pri

In [294]:
print(tabel)

+--------------+----------+-------------+-------------+---------+---------+
|    Gambar    | Accuracy | Sensitivity | Specificity |   Ppv   |   Npv   |
+--------------+----------+-------------+-------------+---------+---------+
| image10prime | 98.0714  |   83.0358   |    99.258   |  89.828 | 98.6692 |
| image11prime | 97.3369  |   78.4974   |   98.4606   | 75.2557 | 98.7142 |
| image13prime | 97.8856  |   81.3118   |   99.2783   | 90.4473 | 98.4428 |
| image14prime | 98.0797  |   81.9453   |    99.594   |  94.986 |  98.327 |
| image15prime | 98.3004  |   80.9634   |   99.4426   | 90.5375 | 98.7546 |
| image16prime | 97.9263  |   75.8992   |   99.6369   |  94.198 | 98.1561 |
| image17prime |  97.549  |   82.0033   |   99.0041   | 88.5161 | 98.3269 |
| image18prime | 97.8589  |   82.8765   |    99.406   | 93.5095 | 98.2523 |
| image19prime | 98.0515  |   78.1241   |   99.6061   | 93.9291 | 98.3155 |
| image1prime  | 98.3561  |   84.6174   |   99.3908   | 91.2744 | 98.8478 |
| image24pri